<a href="https://colab.research.google.com/github/aartikushal/IC-LLAMA-ASSIGNMENT/blob/main/LLAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) Install dependencies**

In [8]:
!pip install transformers torch accelerate

In [ ]:
!huggingface-cli login

**2) Login to Hugging Face**

In [12]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

In [ ]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

**4) Chat helper function**

In [14]:
def generate_llama_response(prompt: str) -> None:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
    )
    print("Chatbot:", sequences[0]['generated_text'])



prompt = 'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n'
generate_llama_response(prompt)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Chatbot: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Sure! If you enjoyed "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Sopranos" - This HBO series is a crime drama that follows the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.
2. "The Wire" - This HBO series explores the drug trade in Baltimore from multiple perspectives, including law enforcement, drug dealers, and politicians. It's known for its gritty realism and complex characters.
3. "Narcos" - This Netflix series tells the true story of Pablo Escobar, the infamous Colombian drug lord, and the DEA agents who hunted him down. It's a thrilling and intense drama that explores the world of drug trafficking and organized crime.
4. "Peaky Blinders" - This BBC series is set in post-World War I


**5) Quick test**

In [15]:
prompt = """I'm a programmer and Python is my favorite language because of it's simple syntax and variety of applications I can build with it.\
Based on that, what language should I learn next?\
Give me 5 recommendations"""
generate_llama_response(prompt)

Chatbot: I'm a programmer and Python is my favorite language because of it's simple syntax and variety of applications I can build with it.Based on that, what language should I learn next?Give me 5 recommendations and explain why you think they're good choices.

1. Java - You should learn Java because it's a popular and versatile language that can be used for developing web applications, Android apps, and enterprise software. Java has a large and active community, which means there are many resources available to help you learn and stay up-to-date with the latest developments. Additionally, Java is known for it's platform independence, which means you can write code that can run on any device that has a Java Virtual Machine (JVM) installed.

2. JavaScript - JavaScript is a must-know language for any web developer. It's used to create interactive web pages, dynamic user interfaces, and web applications. With the rise of front-end development, JavaScript has become even more important, a

In [17]:
prompt = 'How to do quick breakfast using non startchy veggies?\n'
generate_llama_response(prompt)

Chatbot: How to do quick breakfast using non startchy veggies?
Here are some quick breakfast ideas using non-starchy vegetables:

1. Spinach and feta omelette: Whisk together 2 eggs, 1/4 cup fresh spinach leaves, and 1/4 cup crumbled feta cheese. Heat a non-stick pan over medium heat, add the eggs and cook until set. Fold in half and serve.
2. Avocado toast: Toast whole grain bread, mash 1/2 avocado and spread on top. Add a squeeze of lemon juice and a sprinkle of salt and pepper to taste.
3. Broccoli and cheese breakfast burrito: Scramble 2 eggs, add 1/4 cup chopped broccoli florets and 1/4 cup shredded cheese. Wrap in a whole grain tortilla and microwave for 30-45 seconds or until the cheese is melted.
4. Carrot and orange breakfast smoothie: Combine 1/2 cup frozen carrots, 1/


In [16]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "quit", "exit"]:
        print("Chatbot: Goodbye!")
        break
    generate_llama_response(user_input)


You: who is pm of india
Chatbot: who is pm of india

The Prime Minister of India is the head of government of India. The Prime Minister is responsible for leading the country and implementing the policies of the Government of India. The Prime Minister is also the ex-officio chairman of the Indian National Congress and the leader of the Council of Ministers.

The current Prime Minister of India is Narendra Modi, who took office on May 26, 2014. Modi is the first Prime Minister to have been born after India gained independence from British colonial rule. He was re-elected for a second term in the 2019 Indian general election.

The Prime Minister is appointed by the President of India, who is elected indirectly by the members of the Parliament. The Prime Minister is usually the leader of the political party or coalition that has a majority of seats in the Lok Sabha, the lower house of the Indian Parliament.

The Prime Minister has a number of important powers and responsibilities, includi

**6) Gradio Chat UI**

In [18]:

SYSTEM_PROMPT = "You are a helpful, concise assistant."
MAX_TOKENS = 512
TEMPERATURE = 0.7
TOP_P = 0.95

def format_prompt(history, user_message):
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for u, a in history:
        if u:
            messages.append({"role": "user", "content": u})
        if a:
            messages.append({"role": "assistant", "content": a})
    messages.append({"role": "user", "content": user_message})

    # Build chat prompt in LLaMA-2 format
    prompt = ""
    for m in messages:
        if m["role"] == "system":
            prompt += f"<s>[INST] <<SYS>>\n{m['content']}\n<</SYS>>\n"
        elif m["role"] == "user":
            prompt += f"{m['content']} [/INST] "
        elif m["role"] == "assistant":
            prompt += f"{m['content']} </s><s>[INST] "
    return prompt

def generate_chat_response(history, user_message):
    prompt = format_prompt(history, user_message)
    out = llama_pipeline(
        prompt,
        max_new_tokens=MAX_TOKENS,
        do_sample=True,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        eos_token_id=tokenizer.eos_token_id
    )
    reply = out[0]["generated_text"][len(prompt):]
    return reply.strip()

In [19]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# 🦙 LLaMA-2 7B Chatbot")
    chatbot = gr.Chatbot(height=400)
    with gr.Row():
        msg = gr.Textbox(placeholder="Type a message...", scale=8)
        clear = gr.Button("Clear", scale=1)
    temp = gr.Slider(0.0, 1.5, value=TEMPERATURE, step=0.05, label="Temperature")
    top_p = gr.Slider(0.1, 1.0, value=TOP_P, step=0.05, label="Top-p")
    max_new = gr.Slider(64, 1024, value=MAX_TOKENS, step=32, label="Max new tokens")

    def respond(user_message, chat_history, temperature, top_p, max_new_tokens):
        global TEMPERATURE, TOP_P, MAX_TOKENS
        TEMPERATURE = float(temperature)
        TOP_P = float(top_p)
        MAX_TOKENS = int(max_new_tokens)
        bot_message = generate_chat_response(chat_history, user_message)
        chat_history = chat_history + [(user_message, bot_message)]
        return "", chat_history

    msg.submit(respond, [msg, chatbot, temp, top_p, max_new], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()


/tmp/ipython-input-4037629385.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af4c18a3e0f0522d49.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
